In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# ---------------- CONFIG ----------------
input_dir  = r"C:\Users\awais\OneDrive\Desktop\Iran_Iris_Isolated_Enhanced"
output_dir = r"C:\Users\awais\OneDrive\Desktop\Iran_Iris_Isolated_Enhanced_Normalized"
os.makedirs(output_dir, exist_ok=True)

RADIAL_RES, ANGULAR_RES = 64, 512
BG_VALUE = 0  # pure black

# ---------------- HELPERS ----------------
def otsu_foreground_mask(img):
    """Detect iris foreground; treat dark background and remove gray halo."""
    blur = cv2.GaussianBlur(img, (5,5), 0)
    _, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # If background accidentally white, invert
    if np.sum(mask == 255) > np.sum(mask == 0):
        mask = cv2.bitwise_not(mask)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((3,3),np.uint8), 2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8), 2)
    return mask

def remove_gray_background(img):
    """Remove lighter gray around iris and keep only valid texture."""
    hist = cv2.calcHist([img], [0], None, [256], [0,256]).ravel()
    bg_peak = np.argmax(hist[:40])  # background near black
    # dynamic threshold just above background
    thresh = max(bg_peak + 8, 20)
    cleaned = img.copy()
    cleaned[img <= thresh] = BG_VALUE
    return cleaned

def fit_circle(mask):
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None
    c = max(cnts, key=cv2.contourArea)
    (x, y), r = cv2.minEnclosingCircle(c)
    return float(x), float(y), float(r)

def detect_pupil(img, cx, cy, r_iris):
    """Find dark pupil inside the iris."""
    h, w = img.shape
    roi_r = int(0.5 * r_iris)
    x0, y0 = int(max(0, cx - roi_r)), int(max(0, cy - roi_r))
    x1, y1 = int(min(w, cx + roi_r)), int(min(h, cy + roi_r))
    roi = img[y0:y1, x0:x1]
    blur = cv2.GaussianBlur(roi, (5,5), 0)
    _, pmask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    cnts, _ = cv2.findContours(pmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return cx, cy, 0.25 * r_iris
    c = max(cnts, key=cv2.contourArea)
    (px, py), pr = cv2.minEnclosingCircle(c)
    return x0 + px, y0 + py, pr

def rubber_sheet_unwrap(img, cx, cy, r_in, r_out, radial_res=RADIAL_RES, angular_res=ANGULAR_RES):
    """Perform Daugman rubber-sheet normalization."""
    theta = np.linspace(0, 2*np.pi, angular_res, endpoint=False)
    r = np.linspace(0, 1, radial_res)
    xs = (1 - r[:,None]) * (cx + r_in*np.cos(theta)) + r[:,None] * (cx + r_out*np.cos(theta))
    ys = (1 - r[:,None]) * (cy + r_in*np.sin(theta)) + r[:,None] * (cy + r_out*np.sin(theta))
    strip = cv2.remap(img, xs.astype(np.float32), ys.astype(np.float32),
                      interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=BG_VALUE)
    return strip

# ---------------- MAIN ----------------
count = 0
for img_name in tqdm(os.listdir(input_dir), desc="CASIA Isolated → Clean Normalize"):
    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
        continue

    path = os.path.join(input_dir, img_name)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue

    # Step 1 — Remove gray/bright background noise
    img_clean = remove_gray_background(img)

    # Step 2 — Detect iris circle using Otsu mask
    mask = otsu_foreground_mask(img_clean)
    fitted = fit_circle(mask)
    if fitted is None:
        print(f"⚠️ No iris found: {img_name}")
        continue

    cx, cy, r_iris = fitted
    px, py, r_pupil = detect_pupil(img_clean, cx, cy, r_iris)
    r_pupil = max(5, min(r_pupil, 0.8 * r_iris))

    # Step 3 — Mask outside iris explicitly
    yy, xx = np.ogrid[:img_clean.shape[0], :img_clean.shape[1]]
    circle_mask = ((xx - cx)**2 + (yy - cy)**2 <= r_iris**2)
    img_clean[~circle_mask] = BG_VALUE

    # Step 4 — Normalize
    strip = rubber_sheet_unwrap(img_clean, cx, cy, r_pupil, r_iris)

    # Step 5 — Final minor cleanup
    strip = remove_gray_background(strip)

    # Step 6 — Save
    out_path = os.path.join(output_dir, f"{os.path.splitext(img_name)[0]}_norm.png")
    cv2.imwrite(out_path, strip)
    count += 1

print(f"✅ Done. Clean normalized {count} isolated iris images → {output_dir}")


CASIA Isolated → Clean Normalize: 100%|██████████| 792/792 [00:08<00:00, 91.46it/s] 

✅ Done. Clean normalized 792 isolated iris images → C:\Users\awais\OneDrive\Desktop\Iran_Iris_Isolated_Enhanced_Normalized
